# State Farm (no data augmentation)

## Steps:

1. load train and validation data
2. load VGG model
3. remove final layer and precompute
4. replace final layer
5. train
6. test

## 1. Load data

In [3]:
floydhub = False
sample = False

In [4]:
if floydhub:
    data_path = "/input/"
    bottleneck_path = "/output/bottlenecks/"
    results_path = "/output/results/"
else:
    data_path = "./data/"
    if sample:
        data_path += "sample/"
    bottleneck_path = "./bottlenecks/"
    results_path = "./results/"
%mkdir $bottleneck_path
%mkdir $results_path

mkdir: cannot create directory ‘./bottlenecks/’: File exists
mkdir: cannot create directory ‘./results/’: File exists


In [5]:
batch_size = 80

In [6]:
from utils import get_data, get_classes
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(featurewise_center=False, samplewise_center=False)

train_data = get_data(data_path + "train")
valid_data = get_data(data_path + "valid")

Found 19658 images belonging to 10 classes.
Found 2766 images belonging to 10 classes.


In [7]:
val_classes, train_classes, val_labels, train_labels, _,_,_ = get_classes(data_path)

Found 19658 images belonging to 10 classes.
Found 2766 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


## 2. Load VGG

In [ ]:
from vgg16 import Vgg16
from keras.layers.core import Flatten
vgg = Vgg16()

while not isinstance(vgg.model.layers[-1], Flatten):
    vgg.model.pop()

vgg.compile()

In [ ]:
train_bottlenecks = vgg.model.predict(train_data, verbose=1)

 5696/19658 [=======>......................] - ETA: 343s

In [ ]:
valid_bottlenecks = vgg.model.predict(valid_data, verbose=1)

In [ ]:
from utils import save_array
save_array(bottleneck_path+"train_bottlenecks_new.bc", train_bottlenecks)
save_array(bottleneck_path+"valid_bottlenecks_new.bc", valid_bottlenecks)

## 4. Train classification network

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

for layer in vgg.model.layers:
    layer.trainable = False
fc_model = Sequential([
    BatchNormalization(input_shape=train_bottlenecks.shape[1:]),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(10, activation="softmax")
])

fc_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fc_model.summary()

In [17]:
fc_model.optimizer.lr = 0.01
fc_model.fit(train_bottlenecks, train_labels, nb_epoch=10, validation_data=(valid_bottlenecks, val_labels), batch_size=batch_size, verbose=2)

Train on 19658 samples, validate on 2766 samples
Epoch 1/10
34s - loss: 2.5659 - acc: 0.7786 - val_loss: 3.2536 - val_acc: 0.7661
Epoch 2/10
34s - loss: 0.6800 - acc: 0.9464 - val_loss: 3.3381 - val_acc: 0.7722
Epoch 3/10
34s - loss: 0.4817 - acc: 0.9633 - val_loss: 4.5080 - val_acc: 0.6920
Epoch 4/10
34s - loss: 0.3854 - acc: 0.9710 - val_loss: 3.9118 - val_acc: 0.7379
Epoch 5/10
34s - loss: 0.3739 - acc: 0.9733 - val_loss: 3.6164 - val_acc: 0.7570
Epoch 6/10
34s - loss: 0.3139 - acc: 0.9775 - val_loss: 5.0132 - val_acc: 0.6757
Epoch 7/10


KeyboardInterrupt: 

In [18]:
fc_model.optimizer.lr = 0.001
fc_model.fit(train_bottlenecks, train_labels, nb_epoch=10, validation_data=(valid_bottlenecks, val_labels), batch_size=batch_size, verbose=2)

Train on 19658 samples, validate on 2766 samples
Epoch 1/10
34s - loss: 1.2223 - acc: 0.9040 - val_loss: 2.6419 - val_acc: 0.8106
Epoch 2/10
34s - loss: 0.1679 - acc: 0.9854 - val_loss: 2.8714 - val_acc: 0.7874
Epoch 3/10
34s - loss: 0.1506 - acc: 0.9871 - val_loss: 2.8443 - val_acc: 0.8040
Epoch 4/10
34s - loss: 0.1111 - acc: 0.9906 - val_loss: 3.1820 - val_acc: 0.7824
Epoch 5/10
34s - loss: 0.1055 - acc: 0.9910 - val_loss: 5.1031 - val_acc: 0.6565
Epoch 6/10
34s - loss: 0.0990 - acc: 0.9915 - val_loss: 4.2108 - val_acc: 0.7119
Epoch 7/10
34s - loss: 0.1185 - acc: 0.9910 - val_loss: 3.5692 - val_acc: 0.7585
Epoch 8/10
34s - loss: 0.1364 - acc: 0.9893 - val_loss: 3.2212 - val_acc: 0.7842
Epoch 9/10
34s - loss: 0.1978 - acc: 0.9856 - val_loss: 8.5893 - val_acc: 0.4487
Epoch 10/10
34s - loss: 0.2587 - acc: 0.9812 - val_loss: 4.5169 - val_acc: 0.7057


In [19]:
fc_model.optimizer.lr = 0.0001
fc_model.fit(train_bottlenecks, train_labels, nb_epoch=10, validation_data=(valid_bottlenecks, val_labels), batch_size=batch_size, verbose=2)

Train on 19658 samples, validate on 2766 samples
Epoch 1/10
34s - loss: 0.3102 - acc: 0.9784 - val_loss: 3.3973 - val_acc: 0.7795
Epoch 2/10
34s - loss: 0.2523 - acc: 0.9824 - val_loss: 4.9321 - val_acc: 0.6790
Epoch 3/10
35s - loss: 0.2751 - acc: 0.9809 - val_loss: 4.1010 - val_acc: 0.7328
Epoch 4/10
36s - loss: 0.2579 - acc: 0.9828 - val_loss: 3.7483 - val_acc: 0.7603
Epoch 5/10
36s - loss: 0.2853 - acc: 0.9807 - val_loss: 3.9531 - val_acc: 0.7415
Epoch 6/10
36s - loss: 0.2861 - acc: 0.9807 - val_loss: 3.8220 - val_acc: 0.7552
Epoch 7/10
36s - loss: 0.2807 - acc: 0.9814 - val_loss: 3.7130 - val_acc: 0.7636
Epoch 8/10
36s - loss: 0.2566 - acc: 0.9829 - val_loss: 3.4171 - val_acc: 0.7849
Epoch 9/10
37s - loss: 0.2560 - acc: 0.9832 - val_loss: 3.3395 - val_acc: 0.7881
Epoch 10/10
37s - loss: 0.3250 - acc: 0.9787 - val_loss: 3.3976 - val_acc: 0.7838


In [20]:
!sudo ls

bottlenecks  results				     utils.pyc	  vgg16.pyc
data	     State farm VGG 0.ipynb		     vgg16bn.py
download.sh  State farm VGG Data augmentation.ipynb  vgg16bn.pyc
mkdata.py    utils.py				     vgg16.py
